In [1]:
from gensim.corpora import WikiCorpus
from pyhanlp import *
import zipfile,os,sys,re
from pyhanlp.static import download, remove_file, HANLP_DATA_PATH

In [2]:
wikipath=r'D:\NLP\zhwiki-20211101-pages-articles-multistream.xml.bz2'

In [3]:
wiki = WikiCorpus(wikipath, dictionary={})

In [4]:
i=0
p = re.compile(r'[\u4e00-\u9fa5]')
with open('whole.txt','w',encoding='utf-8') as output:
    for text in wiki.get_texts():
        str_temp=''
        for ti in text:
            #temp_i=re.sub("[A-Za-z0-9\!\%\[\]\,\。\s+]", "", i)
            temp_i=''.join(re.findall(p,ti))
            if temp_i:
                str_temp+=temp_i+'\n'
        str_temp=HanLP.tw2s(str_temp)
        #print(str_temp)
        output.write(str_temp)
        i+=1
        if i==10:
            break

E:\Anaconda\envs\py36\lib\site-packages\gensim\utils.py:1332: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected %s; aliasing chunkize to chunkize_serial" % entity)


In [5]:
## 指定 PKU 语料库
PKU98=r'E:\Anaconda\envs\py36\lib\site-packages\pyhanlp\static\data\test\pku98'
PKU199801 = os.path.join(PKU98, '199801.txt')
PKU199801_TRAIN = os.path.join(PKU98, '199801-train.txt')
PKU199801_TEST = os.path.join(PKU98, '199801-test.txt')
POS_MODEL = os.path.join(PKU98, 'pos.bin')
NER_MODEL = os.path.join(PKU98, 'ner.bin')

In [6]:
## 以下开始 CRF 命名实体识别

CRFNERecognizer = JClass('com.hankcs.hanlp.model.crf.CRFNERecognizer')
HMMNERecognizer = JClass('com.hankcs.hanlp.model.hmm.HMMNERecognizer')
NERTrainer = JClass('com.hankcs.hanlp.model.perceptron.NERTrainer')
PerceptronNERecognizer = JClass('com.hankcs.hanlp.model.perceptron.PerceptronNERecognizer')


AbstractLexicalAnalyzer = JClass('com.hankcs.hanlp.tokenizer.lexical.AbstractLexicalAnalyzer')
Utility = JClass('com.hankcs.hanlp.model.perceptron.utility.Utility')
PerceptronSegmenter = JClass('com.hankcs.hanlp.model.perceptron.PerceptronSegmenter')
PerceptronPOSTagger = JClass('com.hankcs.hanlp.model.perceptron.PerceptronPOSTagger')


In [7]:
def train_and_test(model):
    anylizer=AbstractLexicalAnalyzer(PerceptronSegmenter(),PerceptronPOSTagger(),model)
    with open('whole.txt','r',encoding='utf-8') as f:
        whole_any=f.read()
        result=anylizer.analyze(whole_any)
        print('\n'.join(str(result).split('\n')[:20]))
        scores = Utility.evaluateNER(model, PKU199801_TEST)
        Utility.printNERScore(scores)
    return anylizer

In [8]:
def re_train(sent,anylyzer):
    while not anylyzer.analyze(sent.text()).equals(sentence):
        anylyzer.learn(sent)

In [9]:
CRF=CRFNERecognizer()
print('CRF')
crf_any=train_and_test(CRF)

CRF
欧几里得
西元前三世纪/nr 的/u 古希腊/ns 数学家
现在/t 被/p 认为/v 是/v 几何/n 之父
此画/n 为/p 拉斐尔/nrf 的/u 作品
雅典学院
数学
是研究/vn 数量
结构
变化/vn 以及/c 空间/n 等/u 概念/n 的/u 一/m 门/q 学科
从某种/n 角度看/v 属于/v 形式/n 科学/n 的/u 一种
数学/n 利用/v 抽象化/nz 和/c 逻辑推理
从计数
计算
量度
对物体/n 形状/n 及/c 运动/vn 的/u 观察/vn 发展/vn 而成
数学家/n 们/k 拓展/v 这些/r 概念
以公式化/v 新/a 的/u 猜想
以及/vn 从/p 选定/v 的/u 公理/n 及/c 定义/n 出发
严谨/a 地/u 推导/v 出/v 一些/m 定理
对数学/n 基本概念/nz 的/u 完善


In [10]:
HMM=HMMNERecognizer()
HMM.train(PKU199801_TRAIN)
print('HMM')
hmm_any=train_and_test(HMM)

HMM
欧几里得
西元前三世纪/nr 的/u 古希腊/ns 数学家
现在/t 被/p 认为/v 是/v 几何/n 之父
此画/n 为/p 拉斐尔/nrf 的/u 作品
雅典学院
数学
是研究/vn 数量
结构
变化/vn 以及/c 空间/n 等/u 概念/n 的/u 一/m 门/q 学科
从某种/n 角度看/v 属于/v 形式/n 科学/n 的/u 一种
数学/n 利用/v 抽象化/nz 和/c 逻辑推理
从计数
计算
量度
对物体/n 形状/n 及/c 运动/vn 的/u 观察/vn 发展/vn 而成
数学家/n 们/k 拓展/v 这些/r 概念
以公式化/v 新/a 的/u 猜想
以及/vn 从/p 选定/v 的/u 公理/n 及/c 定义/n 出发
严谨/a 地/u 推导/v 出/v 一些/m 定理
对数学/n 基本概念/nz 的/u 完善


In [11]:
NER_temp=NERTrainer()

In [12]:
NER_temp=NERTrainer()
NER=PerceptronNERecognizer(NER_temp.train(PKU199801_TRAIN,NER_MODEL).getModel())
print('NER')
ner_any=train_and_test(NER)

NER
欧几里得
西元前三世纪/nr 的/u 古希腊/ns 数学家
现在/t 被/p 认为/v 是/v 几何/n 之父
此画/n 为/p 拉斐尔/nrf 的/u 作品
雅典学院
数学
是研究/vn 数量
结构
变化/vn 以及/c 空间/n 等/u 概念/n 的/u 一/m 门/q 学科
从某种/n 角度看/v 属于/v 形式/n 科学/n 的/u 一种
数学/n 利用/v 抽象化/nz 和/c 逻辑推理
从计数
计算
量度
对物体/n 形状/n 及/c 运动/vn 的/u 观察/vn 发展/vn 而成
数学家/n 们/k 拓展/v 这些/r 概念
以公式化/v 新/a 的/u 猜想
以及/vn 从/p 选定/v 的/u 公理/n 及/c 定义/n 出发
严谨/a 地/u 推导/v 出/v 一些/m 定理
对数学/n 基本概念/nz 的/u 完善
